In [ ]:
%cd ../

In [ ]:
import time

from plasma.parallel_processing import communicators, queues
from plasma.parallel_processing.communicators.tree_flow import TreeFlow
from plasma.functional import partials, AutoPipe

In [ ]:
def compute(x):
    time.sleep(2)
    return ['a', 'b']

def compute2(x):
    time.sleep(2)
    return x + 'c'

class ComputeClass(AutoPipe):

    def __init__(self, hihi, haha):
        super().__init__()
    
    def run(self, tmp):
        return tmp + 'd'

In [ ]:
out_queue= queues.TransferQueue()

class Flow(TreeFlow):

    def __init__(self):
        super().__init__()

        self.compute = communicators.distributors.IteratorDistributor(compute)
        self.compute2 = compute2
        self.compute3 = ComputeClass(1, 2)
        self.printer1 = partials(print, 'step 1')
        self.printer2 = partials(print, 'step 2')

        self.register_chains(
            (queues.ProcessQueue(5), None, 'compute'),
            (queues.ProcessQueue(5), 'compute', 'compute2'),
            (queues.TransferQueue(), 'compute', 'printer1'),
            (queues.TransferQueue(), 'compute2', 'printer2'),
            (queues.ProcessQueue(), 'compute', 'compute3'),
            (out_queue, 'compute3', None),
        )

flow = Flow()
flow

In [ ]:
with flow:
   for q in flow.inputs.values():
      for i in range(10):
         q.put(i)

In [ ]:
flow.inputs

In [ ]:
flow.outputs

In [ ]:
out_queue.running